# Object Detection Demo

## Demo Overview

In this demonstration we will train a YOLO (You Only Look Once) object detection model to detect the location of turtles and people in images. The scales on the face of sea turtles are unique and can be used to identify individual turtles. If the model detects that a sea turtle is present in an image and generates a bounding box around it, the box can be used to crop out the unnecessary parts of the image.

## Imports Packages, Action Sets and Creates Connection Object

In [ ]:
# Import useful packages

import swat
import seaborn as sns
import pandas as pd
import dlpy
from dlpy import mzmodel
from dlpy.model import DataSpec
from dlpy.applications import YoloV2, Tiny_YoloV2
from dlpy.utils import *

In [ ]:
# Establishes connection to the CAS server

conn = swat.CAS('server.demo.sas.com', 30570, 'student', 'Metadata0')

In [ ]:
# Changes session timeout

mytime = 60*60*24
conn.session.timeout(time=mytime)

In [ ]:
# Loads the necessary action sets

conn.loadactionset("image")
conn.loadactionset("deeplearn")
conn.loadactionset("sampling")

## Object Detection Table Creation and Exploration

In [ ]:
# # Creates the path for the local annotations and the images

# img_path  = "/path/to/images/and/annotations" 

# # Creates the object detection table

# obj_det_targets = create_object_detection_table(conn,
#                                                 data_path = img_path,
#                                                 local_path = img_path,
#                                                 coord_type = "yolo",
#                                                 image_size = (640, 640),
#                                                 output = "dr_taco")

In [ ]:
# Creates a caslib pointing to the directory where the dataset is located

conn.table.addcaslib(name = "mycl", path = "/workshop/winsas/VOSI", subdirectories = True)

In [ ]:
# Loads the object detection table 

conn.table.loadtable(path = "galapagos_objdet.sashdat", caslib = "mycl", casout = dict(name = "galapagos", replace = True))

In [ ]:
# Displays a sample of the uploaded galapagos dataset

galapagos = conn.CASTable("galapagos")
galapagos.head()

In [ ]:
# Displays the detected object types

galapagos["_Object0_"].value_counts()

In [ ]:
# Creates an image table of the galapagos dataset and displays a summary of the results

galapagos_img_tbl = dlpy.images.ImageTable.from_table(galapagos)

galapagos_img_tbl.image_summary

In [ ]:
# Stores all of the columns so that this information can be copied freely as new CAS Tables are created

columns = galapagos.columns.to_list()

In [ ]:
# Displays a sample of the detection objects

display_object_detections(conn,
                          "galapagos",
                          coord_type = "yolo",
                          num_plot = 5,
                          n_col = 5
                          )

## Data Partitioning

In [ ]:
# Partitions the data into training and validation

conn.sampling.srs(table=dict(name="galapagos"), samppct=90, seed=42, partind=True,
                  output=dict(casout=dict(name="sampled_data", replace=True), 
                              copyvars="all", partindname="PartInd")
                 )

In [ ]:
# Partitions the data into training and validation

train = conn.CASTable("sampled_data").query("PartInd = 1 and _nObjects_ > 0")
valid = conn.CASTable("sampled_data").query("PartInd = 0 and _nObjects_ > 0")

In [ ]:
# Displays the shapes of the partitions 

display(train.shape)
display(valid.shape)

In [ ]:
# Ensures that partitions are correct

display(train["PartInd"].unique())
display(valid["PartInd"].unique())

### Data Augmentation

In [ ]:
# Applies image augmentation to the training datasets
    
conn.image.augmentImages(table = train, casout = {"name": "aug_train", "replace": True},  # output CASTable
                         croplist=[{
                                    "usewholeimage": True,
                                    "mutations": {
                                        "sharpen": True,
                                        "lighten": True,
                                        "darken": True,
                                        "horizontalFlip": True,
                                        "colorshifting": True,
                                        "colorjittering": True,
                                        "colorshifting": True,
                                        "colorjittering": True,                                      
                                        }
                                    }
                                   ],
                         copyVars=columns,
                         writeRandomly=True,
                         seed=42
                         )

In [ ]:
# Creates a CASTable object pointing to the in-memory table

aug_train = conn.CASTable("aug_train")
aug_train.head()

In [ ]:
# Displays a sample of the augmented images

display_object_detections(conn, aug_train, coord_type = "yolo",num_plot = 5, n_col = 5)

In [ ]:
# Appends the augmented data to the training dataset

conn.deeplearn.dljoin(table = train, annotatedtable = aug_train, jointype = "append",
                      casout = dict(name = "merged_train", replace = True))

# Shuffles the dataset

conn.table.shuffle(table=dict(name = "merged_train"), casout = dict(name = "merged_train", replace = True))

# Creates a CASTable object pointing to the in-memory table

merged_train = conn.CASTable("merged_train")
print("The merged_train dataset contains", str(merged_train.shape[0]), "images.")

In [ ]:
# # Alternate way of applying augmentations, image actionset has more options but this way is easier

# aug_train_img_tbl = dlpy.images.ImageTable.from_table(aug_train)
# aug_train_img_tbl.random_mutations(sharpen = True, lighten = True, darken = True, horizontal_flip = True, color_shift = True, color_jitter = True, color_shift = True)

## Anchor Boxes

In [ ]:
# Generates the anchors that will be used for the TinyYOLOv2 model using only the training data set

anchors = get_anchors(conn, data = merged_train, coord_type = "yolo", n_anchors = 5)

## Optimization Settings

In [ ]:
# Specifies the optimization algorithm details 

optimizer = dict(miniBatchSize=8,
                 logLevel=2,
                 maxEpochs=5,
                 totalBudget=200,
                 regL1=dict(lowerBound=0, upperBound=0.01, logScale=True),
                 regL2=dict(lowerBound=0.0001, upperBound=0.01, logScale=True),
                 algorithm=dict(method="ADAM", 
                                clipgradmax=100,
                                clipgradmin=-100,
                                learningRatePolicy="POLY",
                                learningRate=dict(lowerBound=0.000001, upperBound=0.0001, logScale=True),
                                beta1=dict(lowerBound=0.8, upperBound=0.999),
                                beta2=dict(lowerBound=0.8, upperBound=0.999),
                                ),
                 stagnation=3,  # early stopping
                 threshold=1e-6,
                 ignoreTrainingError=True,
                 dropout=dict(lowerBound=0.0, upperBound=0.20),
                 dropoutType="INVERTED",
                 )

## YOLOv2 Model Definition

In [ ]:
# Defines the YOLO model object

yolo = YoloV2(conn, n_classes = 2, anchors = anchors, n_channels = 3, model_table = "galapagos", 
              iou_threshold=0.2, detection_threshold = 0.35, class_scale = 1, coord_scale = 1, prediction_not_a_object_scale = 0.5)

In [ ]:
# Displays the model architecture, along with layer names, types, and input/output shapes

yolo.print_summary()

## Pre-Trained DarkNet Weights

Deep Learning model weights can be downloaded from the <a href = "https://support.sas.com/documentation/prod-p/vdmml/zip/index.html">SAS Documentation</a> for a variety of popular deep learning models.

In [ ]:
# Loads table containing pre-trained weights

conn.table.loadtable(path = "darknet_weights.sashdat", caslib = "mycl", casout = dict(name = "darknet", replace = True))

In [ ]:
# Attaches the weights to the yolo model object

yolo.set_weights("darknet")

## YOLOv2 Training

In [ ]:
# Creates a list of the model targets

obj_det_targets = ["_nObjects_", "_Object0_", "_Object0_x", "_Object0_y", "_Object0_width", "_Object0_height",
                   "_Object1_", "_Object1_x", "_Object1_y", "_Object1_width", "_Object1_height",
                   "_Object2_", "_Object2_x", "_Object2_y", "_Object2_width", "_Object2_height"]

In [ ]:
# Specifies the necessary model input information using the dataspecs option

data_specs = [DataSpec(type_='IMAGE', layer = 'Input1', data = "_image_"),
              DataSpec(type_='OBJECTDETECTION', layer = 'Detection1', data = obj_det_targets)]

In [30]:
# # Trains and tunes the YOLOv2 model with darknet as weights as the starting point

# yolo.tune(data = train, validtable = valid, dataspecs = data_specs, 
#           optimizer = optimizer, gpu = 1,
#           initWeights = yolo.model_weights,
#           modelWeights = dict(name = "yolo_weights", replace = True),
#           seed = 42)

NOTE: Using controller.sas-cas-server-default.edu.svc.cluster.local: 1 out of 1 available GPU devices.
NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 19853216.
NOTE:  Loading weights cost       0.08 (s).
NOTE:  Initializing each layer cost       4.33 (s).
NOTE:  The total number of threads on each worker is 32.
NOTE:  The maximum mini-batch size per thread on each worker is 8.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 256.
NOTE: Evaluating 10 hyperparameter configurations.
NOTE: Tuning learningRate with lower bound   1E-6 and upper bound 0.0001 using logarithmic scale.
NOTE: Tuning regL1 with lower bound      0 and upper bound   0.01 using logarithmic scale.
NOTE: Tuning regL2 with lower bound 0.0001 and upper bound   0.01 using logarithmic scale.
NOTE: Tuning dropOut with lower bound      0 and upper bound    0.2
NOTE: Tuning beta1 with lower bound    0.8 and upper bound  0.999
NOTE: Tuning beta2 with lower bound   

,Descr,Value
0,Model Name,galapagos
1,Model Type,Convolutional Neural Network
2,Number of Layers,44
3,Number of Input Layers,1
4,Number of Output Layers,0
5,Number of Convolutional Layers,19
6,Number of Pooling Layers,5
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,18
9,Number of Detection Layers,1


In [ ]:
# # Saves the model table, weights, and weight attributes

# yolo.save_to_table("/workshop/winsas/VOSI")

In [ ]:
# Loads model weights and their attributes onto the model object

yolo.load("/workshop/winsas/VOSI/galapagos.sashdat") 

## YOLOv2 Scoring

In [ ]:
# Check existing model weights

yolo.model_weights

In [ ]:
# Attaches tuned weights

yolo.set_weights("yolo_weights")

In [ ]:
# Generates predictions using the YOLO model 

yolo.predict(valid, gpu = 1)

## Predictions Table Modification

In [ ]:
# Creates an object to reference the prediction table

pred_tbl = yolo.valid_res_tbl
pred_tbl.shape

In [ ]:
# Displays the columns of the predictions table

pred_tbl.columns

In [ ]:
# Stores and subsets the columns so that up to three objects can be identified

pred_cols = pred_tbl.columns.to_list()[:19]

In [ ]:
# Makes a copy of the predictions table

conn.table.copytable(table = pred_tbl.name, casout = dict(name = "narrow_pred", replace = True))
conn.table.altertable(name = "narrow_pred", keep = pred_cols)

narrow_pred = conn.CASTable("narrow_pred")
narrow_pred.head()

In [ ]:
# Makes a copy of the validation table

conn.table.copytable(table = valid, casout = dict(name = "narrow_valid", replace = True))
conn.table.altertable(name = "narrow_valid", keep = ["idjoin", "_image_", "nObjects", "Object0", "Object1", "Object2"], 
                      columns = [dict(name = "_nObjects_", rename = "nObjects"),
                                 dict(name = "_Object0_", rename = "Object0"), 
                                 dict(name = "_Object1_", rename = "Object1"), 
                                 dict(name = "_Object2_", rename = "Object2")])

narrow_valid = conn.CASTable("narrow_valid")
narrow_valid.head()

In [ ]:
# Appends validation data to the predictions table 

conn.deeplearn.dljoin(table = narrow_valid, annotatedtable = narrow_pred, 
                      id = "idjoin",
                      casout = dict(name = "pred_w_labels", replace = True))

In [ ]:
# Creates a CAS object that points at the new predictions table

preds_final = conn.CASTable("pred_w_labels")
preds_final.head()

## YOLOv2 Assessment

In [ ]:
# Evaluates the YOLO model

yolo_eval = yolo.evaluate_object_detection(valid, "YOLO")
yolo_eval

In [ ]:
# Explores the evaluations at different IoU thresholds

eval_data = [(i["IoU Threshold"], i["AP"], 
              i["Class Evaluation"][0]["total TP"]/i["Class Evaluation"][0]["total positives"],
              i["Class Evaluation"][1]["total TP"]/i["Class Evaluation"][1]["total positives"]) for i in yolo_eval]
eval_df = pd.DataFrame(data = eval_data, columns = ["IoU", "AP", "Person Recall", "Turtle Recall"])
eval_df.head(10)

In [ ]:
# Plots the AP and Recall vs. IoU Threshold


sns.lineplot(data = eval_df, x = "IoU", y = "AP")
sns.lineplot(data = eval_df, x = "IoU", y = "Turtle Recall").set_title("AP and Recall for Various IoU Thresholds")

In [ ]:
# Displays a sample of the predictions

display_object_detections(conn, pred_tbl, coord_type = "yolo", num_plot = 5, n_col = 5)

In [ ]:
# Displays the number of matches in the predictions and validation datasets

matches = preds_final.query("nObjects = _nObjects_")
print("There are", str(matches.shape[0]), "correctly assessed images out of", str(pred_tbl.shape[0]),"validation images.")

In [ ]:
# Displays the number of mismatches in the predictions and validation datasets

mismatches = preds_final.query("Object0 ^= _Object0_ and _nObjects_ > 0")
print("There are", str(mismatches.shape[0]), "mismatches between the first predicted objects.")

In [ ]:
# Displays a sample of some of the mismatches

mismatches.head(10)

In [ ]:
# Displays some of the mismatches 

display_object_detections(conn, mismatches, coord_type = "yolo", n_col = 5, num_plot = 15)

### Saving Model

In [ ]:
# # Model can be saved as an astore file 
# # Requires client-side path

# yolo.save_to_astore("D:\\galapagos\\yolo_model")

In [ ]:
# # Saving model as table
# # Requires server-side path and also stores weights and attributes

# yolo.save_to_table("/workshop/winsas/VOSI")

In [ ]:
# Ends the session

conn.session.endsession()